In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import VarianceRatio
from hurst import compute_Hc
from statsmodels.tsa.stattools import kpss
import statsmodels.api as sm


In [ ]:
# download S&P 500 data pricing hystory for 5 years

sp500 = yf.download('^GSPC', period='5y')
sp500.head()

In [ ]:
# sliding mean analysis

sp500['SMA_50'] = sp500['Close'].rolling(window=50).mean()
sp500['SMA_200'] = sp500['Close'].rolling(window=200).mean()

# Plotting the closing price and the moving averages
plt.figure(figsize=(14, 7))
plt.plot(sp500['Close'], label='S&P 500 Close Price', color='blue')
plt.plot(sp500['SMA_50'], label='50-Day SMA', color='orange')
plt.plot(sp500['SMA_200'], label='200-Day SMA', color='red')
plt.title('S&P 500 Closing Price and Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# compute the daily returns
sp500['Daily Return'] = sp500['Close'].pct_change()
print(sp500['Daily Return'].describe())
# Plotting the daily returns
plt.figure(figsize=(14, 7))
plt.plot(sp500['Daily Return'], label='Daily Return', color='green')
plt.title('S&P 500 Daily Returns')
plt.xlabel('Date')
plt.ylabel('Daily Return')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# compute cumuilative returns
sp500['Cumulative Return'] = (1 + sp500['Daily Return']).cumprod()
print(sp500['Cumulative Return'].describe())
# Plotting the cumulative returns
plt.figure(figsize=(14, 7))
plt.plot(sp500['Cumulative Return'], label='Cumulative Return', color='purple')
plt.title('S&P 500 Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# compute rolling volatility
sp500['Rolling Volatility'] = sp500['Daily Return'].rolling(window=21).std() * (252 ** 0.5)  # Annualized volatility
print(sp500['Rolling Volatility'].describe())

# Plotting the rolling volatility and MAD
plt.figure(figsize=(14, 7))
plt.plot(sp500['Rolling Volatility'], label='Rolling Volatility (Annualized)', color='brown')
plt.title('S&P 500 Rolling Volatility (Annualized)')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid()
plt.show()  

In [ ]:
# compute MAD and quantiles
sp500['MAD'] = sp500['Close'].rolling(window=50).apply(lambda x: (x - x.mean()).abs().mean())
print(sp500['MAD'].describe())
# Plotting the Mean Absolute Deviation (MAD)
plt.figure(figsize=(14, 7))
plt.plot(sp500['MAD'], label='Mean Absolute Deviation (MAD)', color='cyan')
plt.title('S&P 500 Mean Absolute Deviation (MAD)')
plt.xlabel('Date')
plt.ylabel('MAD')
plt.legend()
plt.grid()
plt.show()      

### Strategy 0:  Moving Average Crossover with Persistent Position

In this implementation, we interpret the trading signals as entry and exit instructions, rather than one-day actions.

Buy Signal (Signal = +1):
Enter a long position when the 20-day moving average crosses above the 50-day moving average.

Position Persistence:
After entering, remain in the long position continuously (carry it forward) until a sell signal is generated.

Sell Signal (Signal = -1):
Exit the position (return to cash exposure) when the 20-day moving average crosses back below the 50-day moving average.

Implementation:

We initialize all positions to zero.
When a buy signal occurs, the position is set to 1 (long).
We use .ffill() to carry the position forward over time.
When a sell signal occurs, the position resets to 0.

The strategy returns are computed by multiplying the lagged position by the daily returns, simulating entering positions at the close following the signal.

This approach is designed to capture sustained trends, remaining invested for the entire period between signals, rather than reacting to only the immediate day after each crossover.



In [ ]:
# compute 5-day, 21-day and 63-day returns
sp500['5-Day Return'] = sp500['Close'].pct_change(periods=5)
sp500['21-Day Return'] = sp500['Close'].pct_change(periods=21)
sp500['63-Day Return'] = sp500['Close'].pct_change(periods=63)

#compute 20-day and 50-day moving average
sp500['20-Day MA'] = sp500['Close'].rolling(window=20).mean()
sp500['50-Day MA'] = sp500['Close'].rolling(window=50).mean()

sp500['Signal'] = 0
sp500.loc[sp500['20-Day MA'] > sp500['50-Day MA'], 'Signal'] = 1
sp500.loc[sp500['20-Day MA'] < sp500['50-Day MA'], 'Signal'] = -1

# Plotting the 20-day and 50-day moving averages with buy/sell signals
plt.figure(figsize=(14, 7))
plt.plot(sp500['Close'], label='S&P 500 Close Price', color='blue')
plt.plot(sp500['20-Day MA'], label='20-Day MA', color='orange')
plt.plot(sp500['50-Day MA'], label='50-Day MA', color='red')
plt.scatter(sp500.index[sp500['Signal'] == 1], sp500['Close'][sp500['Signal'] == 1], label='Buy Signal', marker='^', color='green', s=100)
plt.scatter(sp500.index[sp500['Signal'] == -1], sp500['Close'][sp500['Signal'] == -1], label='Sell Signal', marker='v', color='red', s=100)
plt.title('S&P 500 Moving Averages with Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid()
plt.show()  

In [ ]:
"""
A simple moving average crossover trading strategy and evaluate its performance 
relative to a passive buy-and-hold approach.
"""

# compute position based on the signals

sp500['Position'] = 0
sp500.loc[sp500['Signal'] == 1, 'Position'] = 1
sp500['Position'] = sp500['Position'].ffill()  # carry forward position
sp500.loc[sp500['Signal'] == -1, 'Position'] = 0

# compute cumulative returns for buy and hold strategy
sp500['Cumulative BuyHold'] = (1 + sp500['Daily Return']).cumprod()

# compute strategy returns based on the position
sp500['Strategy0_Return'] = sp500['Position'].shift(1) * sp500['Daily Return']
# compute cumulative strategy returns
sp500['Cumulative_Strategy0'] = (1 + sp500['Strategy0_Return']).cumprod()

# Plotting the strategy returns
plt.figure(figsize=(12, 6))
plt.plot(sp500['Cumulative_Strategy0'], label='Strategy 0 Returns', color='blue')
plt.plot(sp500['Cumulative BuyHold'], label='Buy & Hold Returns', color='orange')
plt.title('Cumulative Returns: Strategy 0 vs. Buy & Hold')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid()
plt.show()  




### Performance metrics for Strategy 0

This section calculates key performance indicators (KPIs) for the strategy and compares them to a simple buy-and-hold benchmark.

####  `strategy_return`
```python
strategy_return = sp500['Cumulative_Strategy0'].iloc[-1] - 1
```
The total return of the strategy over the full backtest period (as a percentage). We subtract 1 to convert cumulative return into net gain.

####  `buy_hold_return`
```python
buy_hold_return = sp500['Cumulative BuyHold'].iloc[-1] - 1
```
Same as above, but for the passive buy-and-hold benchmark.

---

####  `annualized_strategy_return`
```python
annualized_strategy_return = (1 + strategy_return) ** (252 / len(sp500)) - 1
```
Annualized return of the strategy, assuming compounding.  
- `252` is the number of trading days in a year.  
- We adjust for the actual length of the dataset with `len(sp500)`.

####  `annualized_buy_hold_return`
```python
annualized_buy_hold_return = (1 + buy_hold_return) ** (252 / len(sp500)) - 1
```
Same as above, but for the buy-and-hold approach.

---

####  `annualized_volatility`
```python
annualized_volatility = sp500['Daily Return'].std() * (252 ** 0.5)
```
Measures the standard deviation (volatility) of daily returns, scaled to a full year using the square root of 252.

---

####  `sharpe_ratio`
```python
sharpe_ratio = annualized_strategy_return / annualized_volatility
```
The Sharpe ratio evaluates **risk-adjusted return**:
- Higher is better.
- It tells us how much return we earn for each unit of risk taken.
- Assumes a risk-free rate of 0%.

---

###  `max_drawdown`

```python
max_drawdown = (sp500['Cumulative_Strategy0'] / sp500['Cumulative_Strategy0'].cummax() - 1).min()
```

**Maximum drawdown** measures the worst-case historical loss from a peak in cumulative return to a following trough. This risk metric reflects how painful the strategy would have been to hold during its worst period.

A lower (less negative) max drawdown indicates a smoother equity curve and less downside risk.
---

These metrics give a clearer picture of how the strategy performs not just in absolute terms, but relative to its risk and to a benchmark.


In [ ]:
# function to compute metrics for the strategy
def strategy_metrics(cumulative_returns, cumulative_buy_hold):
    strategy_return = cumulative_returns.iloc[-1] - 1
    buy_hold_return = cumulative_buy_hold.iloc[-1] - 1
    annualized_strategy_return = (1 + strategy_return) ** (252 / len(cumulative_returns))
    annualized_buy_hold_return = (1 + buy_hold_return) ** (252 / len(cumulative_buy_hold))
    annualized_volatility = cumulative_returns.pct_change().std() * (252 ** 0.5)
    sharpe_ratio = annualized_strategy_return / annualized_volatility   
    max_drawdown = (cumulative_returns / cumulative_returns.cummax() - 1).min()
    
    return {
        'strategy_return': strategy_return,
        'buy_hold_return': buy_hold_return,
        'annualized_strategy_return': annualized_strategy_return,
        'annualized_buy_hold_return': annualized_buy_hold_return,
        'annualized_volatility': annualized_volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown
    }

# compute the performance metrics for strategy 0
metrics = strategy_metrics(sp500['Cumulative_Strategy0'], sp500['Cumulative BuyHold'])

# Print the performance metrics values
print(f"Strategy 0 Return: {metrics['strategy_return']:.2%}")
print(f"Buy & Hold Return: {metrics['buy_hold_return']:.2%}")
print(f"Annualized Strategy 0 Return: {metrics['annualized_strategy_return']:.2%}")
print(f"Annualized Buy & Hold Return: {metrics['annualized_buy_hold_return']:.2%}")
print(f"Annualized Volatility: {metrics['annualized_volatility']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")

### Strategy 0 Performance Discussion

As the cumulative returns plot clearly shows, this simple moving average crossover strategy underperforms the buy-and-hold benchmark—by quite a margin.

A few likely reasons for that:

- The strategy is slow to react: it often enters late and exits early, missing out on the core of an uptrend.
- It tends to get whipsawed in choppy markets—exiting on minor pullbacks and re-entering too late.
- And of course, over this period, the S&P 500 has been on a relatively strong and consistent uptrend—so staying invested without trying to time the market was actually the better choice.

That said, I think this still serves as a good base. It highlights how important it is to:

- Evaluate strategies on different market regimes (not just bull runs),
- Consider out-of-sample performance seriously,
- And maybe think about adding a few improvements—like a volatility filter, or replacing moving averages with something more responsive (RSI, MACD, or even basic price momentum).

Even if this version doesn’t “beat the market,” it sets up a clean, testable framework for exploring new ideas.


In [ ]:
# adding transaction costs
transaction_cost = 0.001  # 0.1% transaction cost
sp500['Transaction Cost'] = transaction_cost * sp500['Position'].diff().abs()
sp500['Strategy0_Return_With_Costs'] = sp500['Strategy0_Return'] - sp500['Transaction Cost']
sp500['Cumulative_Strategy0_With_Costs'] = (1 + sp500['Strategy0_Return_With_Costs']).cumprod()   

# Plotting the strategy returns with transaction costs
plt.figure(figsize=(12, 6))
plt.plot(sp500['Cumulative_Strategy0_With_Costs'], label='Strategy 0 Returns with Costs', color='purple')
plt.plot(sp500['Cumulative_Strategy0'], label='Strategy 0 Returns without Costs', color='blue')
plt.plot(sp500['Cumulative BuyHold'], label='Buy & Hold Returns', color='orange')
plt.title('Cumulative Returns: Strategy 0 vs. Buy & Hold (with Costs)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()    

In [ ]:
# ADF test for stationarity
print("==== Augmented Dickey-Fuller Test on Returns====")
adf_result = adfuller(sp500['Daily Return'].dropna())
adf_stat, adf_p = adf_result[0], adf_result[1]
print(f"ADF Statistic: {adf_stat:.4f}")
print(f"p-value: {adf_p:.4f}")

if adf_p < 0.05:
    print("Result: The time series is likely **stationary** (reject null hypothesis).")
else:
    print("Result: The time series appears **non-stationary** (fail to reject null hypothesis).")

# Variance ratio test
print("\n==== Variance Ratio Test on Returns ====")
vr_test = VarianceRatio(sp500['Daily Return'].dropna(), lags=2)
vr_stat, vr_p = vr_test.stat, vr_test.pvalue
print(f"Variance Ratio: {vr_stat:.4f}")
print(f"p-value: {vr_p:.4g}")

if vr_p < 0.05:
    if vr_stat < 0:
        print("Result: The series shows **mean reversion** (reject random walk null hypothesis).")
    else:
        print("Result: The series shows **trend-following behavior** (reject random walk null hypothesis).")
else:
    print("Result: The series may follow a **random walk** (fail to reject null hypothesis).")

#Hurst exponent 
print("\n==== Hurst Exponent ====")
hurst_stat, _, _ = compute_Hc(sp500['Daily Return'].dropna(), kind='change', simplified=True)
print(f"Hurst Exponent: {hurst_stat:.4f}")  
if hurst_stat < 0.5:
    print("Result: The series shows **mean reversion** behavior.")
elif hurst_stat == 0.5:
    print("Result: The series follows a **random walk**.")
else:
    print("Result: The series shows **trending behavior**.")

# KPSS test for stationarity
print("\n==== KPSS Test on Returns ====")
kpss_result = kpss(sp500['Daily Return'].dropna(), regression='c')
kpss_stat, kpss_p = kpss_result[0], kpss_result[1]
print(f"KPSS Statistic: {kpss_stat:.4f}")
print(f"p-value: {kpss_p:.4f}") 



### Statistical Summary: Return Series Diagnostics

This section evaluates the statistical properties of the **daily returns** of the S&P 500. Several standard tests are used to assess **stationarity**, **mean-reversion**, and **memory structure** of the time series.

---

#### **Augmented Dickey-Fuller (ADF) Test**
- **Result**: ADF Statistic = -21.78, p-value ≈ 0.0000  
- **Interpretation**: Reject the null hypothesis of a unit root.  
- The return series is **stationary**, as expected for financial returns.

---

#### **Variance Ratio Test**
- **Result**: Variance Ratio = -8.12, p-value ≈ 0.0000  
- **Interpretation**: Reject the null hypothesis of a random walk.  
- The return series shows signs of **mean reversion**, suggesting short-term reversals.

---

#### **Hurst Exponent**
- **Result**: H = 0.5815  
- **Interpretation**: Suggests weak **trend-following behavior** (H > 0.5).  
- May indicate some **short-term autocorrelation**, though results are close to neutral.

---

#### **KPSS Test**
- **Result**: KPSS Statistic = 0.0879, p-value > 0.10  
- **Interpretation**: Fail to reject the null hypothesis of stationarity.  
- **Confirms** ADF result: the series is **stationary**.

---

### Overall Conclusion
The daily return series is **stationary** and exhibits some **mean-reversion**, with a hint of weak trend persistence. These properties are typical of financial return series and offer useful structure for further modeling or strategy development.



In [ ]:
# check if price data is stationary
# ADF test for stationarity on price data
print("==== Augmented Dickey-Fuller Test on Price ====")
adf_result = adfuller(sp500['Close'].dropna())
adf_stat, adf_p = adf_result[0], adf_result[1]
print(f"ADF Statistic: {adf_stat:.4f}")
print(f"p-value: {adf_p:.4f}")      
if adf_p < 0.05:
    print("Result: The price series is likely **stationary** (reject null hypothesis).")
else:
    print("Result: The price series appears **non-stationary** (fail to reject null hypothesis).")      
    
# Variance ratio test on price data
print("\n==== Variance Ratio Test on Price ====")
vr_test = VarianceRatio(sp500['Close'].dropna(), lags=2)
vr_stat, vr_p = vr_test.stat, vr_test.pvalue
print(f"Variance Ratio: {vr_stat:.4f}")
print(f"p-value: {vr_p:.4g}")   

if vr_p < 0.05:
    if vr_stat < 0:
        print("Result: The price series shows **mean reversion** (reject random walk null hypothesis).")
    else:
        print("Result: The price series shows **trend-following behavior** (reject random walk null hypothesis).")
else:
    print("Result: The price series may follow a **random walk** (fail to reject null hypothesis).")    
  
  
#Hurst exponent for price data
print("\n==== Hurst Exponent on Price ====")
H, c, data_reg = compute_Hc(sp500['Close'].dropna(), kind='price', simplified=True)
print(f"Hurst Exponent: {H:.4f}")
if H < 0.5:
    print("Result: The price series shows **mean reversion** behavior.")
elif H > 0.5:
    print("Result: The price series shows **trend-following** behavior.")
else:
    print("Result: The price series is likely a **random walk**.")  
    
# generalized Hurst exponent for log price data
print("\n==== Generalized Hurst Exponent on log Price ====")        
log_prices = sp500['Close'].dropna().apply(lambda x: np.log(x))
H, c, data_reg = compute_Hc(log_prices, kind='price', simplified=False)
print(f"Generalized Hurst Exponent: {H:.4f}")
if H < 0.5:
    print("Result: The log price series shows **mean reversion** behavior.")
elif H > 0.5:
    print("Result: The log price series shows **trend-following** behavior.")
else:
    print("Result: The log price series is likely a **random walk**.")
        
# KPSS test for stationarity on price data
print("\n==== KPSS Test on Price ====")
kpss_result = kpss(sp500['Close'].dropna(), regression='c')
kpss_stat, kpss_p = kpss_result[0], kpss_result[1]
print(f"KPSS Statistic: {kpss_stat:.4f}")
print(f"p-value: {kpss_p:.4f}") 

### Statistical Summary: Price Series Diagnostics

This section evaluates the statistical properties of the **S&P 500 price series** using several tests to assess **stationarity**, **random walk behavior**, and **memory structure**.

---

#### **Augmented Dickey-Fuller (ADF) Test**
- **Result**: ADF Statistic = –0.5152, p-value = 0.8890  
- **Interpretation**: Fail to reject the null hypothesis of a unit root.  
- The price series is **non-stationary**, showing no tendency to revert to a long-term mean.

---

#### **Variance Ratio Test**
- **Result**: Variance Ratio = –0.4676, p-value = 0.6401  
- **Interpretation**: Fail to reject the null hypothesis of a random walk.  
- The price series may follow a **random walk**, consistent with financial theory.

---

#### **Hurst Exponent**
- **Result**: H = 0.6431  
- **Interpretation**: H > 0.5 **trend-following behavior**.  
- Prices show persistence, i.e., upward moves tend to be followed by further upward moves.

---
#### **Generalized Hurst Exponent on Log-transformed data**
- **Result**: Hg = 0.5458
- **Interpretation**: Hg > 0.5 **persistent (trend-following) behavior**.  
- The log-transformed price series demonstrates long-term memory, suggesting directional moves tend to persist. This is consistent with a market that trends rather than mean-reverts.  

---

#### **KPSS Test**
- **Result**: KPSS Statistic = 4.4305, p-value < 0.01  
- **Interpretation**: Reject the null hypothesis of stationarity.  
- The price series is strongly **non-stationary**, confirming ADF result.

> Warning note: The KPSS warning indicates the test statistic is far beyond the table's range — this reinforces the rejection of stationarity.

---

### Overall Conclusion

The S&P 500 price series is **non-stationary**, likely follows a **random walk**, and exhibits **trend-following tendencies**. For modeling or forecasting, it is standard practice to transform the price into **returns** or **log returns**, which are more likely to be stationary and statistically well-behaved.


In [ ]:
def halflife(series):
    lagged_series = series.shift(1).dropna()
    delta = series.diff().dropna()

    # Align lengths
    lagged_series = lagged_series.loc[delta.index]

    # Regress delta on lagged series
    model = sm.OLS(delta, sm.add_constant(lagged_series)).fit()
    beta = model.params.iloc[1]  # Coefficient of the lagged series

    halflife = -np.log(2) / beta if beta != 0 else np.nan
    return halflife

halflife_value = halflife(sp500['Close'].dropna())
print(f"Halflife of Mean Reversion: {halflife_value:.2f} days")

### **Halflife of Mean Reversion**
- **Result**: `505.62 days`  
- **Interpretation**: This value estimates how long it takes for a shock to the price series to decay halfway back to its mean, assuming mean reversion is present.
- **Insight**: Compared to the ~115 days halflife reported in the Chan "Algorithmic trading" (for USD.CAD), this result is **much longer**, suggesting that the S&P 500 price series is **weakly mean-reverting or nearly non-reverting**.
- **Implication**: For short- to medium-term trading strategies, this halflife might be **too long** to act on directly. However, it helps define a meaningful **look-back window** and informs **holding period expectations** for strategies based on mean reversion.


### Strategy 1: Linear Mean Reversion 


In [ ]:
# strategy 1: linear mean reversion

lookback = round(halflife_value)  # Use the halflife as the lookback period

# Step 1: Compute rolling mean and std over the halflife window
rolling_mean = sp500['Close'].rolling(window=lookback).mean()
rolling_std = sp500['Close'].rolling(window=lookback).std()

# Step 2: Compute z-score (normalized deviation from mean)
z_score = -(sp500['Close'] - rolling_mean) / rolling_std

# Step 3: Market value (position) = -z_score
mkt_val = z_score.shift(1)  # lagging by 1 day (to avoid lookahead bias)

# Step 4: Daily returns of the price
daily_return = sp500['Close'].pct_change()

# Step 5: Strategy P&L = position * next day return
strategy_pnl = mkt_val * daily_return

# Optional: cumulative return
cumulative_return = (1 + strategy_pnl).cumprod()
cumulative_return = cumulative_return.squeeze()

# Display final stats

plt.figure(figsize=(10, 4))
cumulative_return.plot(title='Cumulative Return of Mean-Reversion Strategy')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()

metrics = strategy_metrics(cumulative_return, sp500['Cumulative BuyHold'])
# Print the performance metrics values
print(f"Strategy 1 Return: {metrics['strategy_return']:.2%}")
print(f"Buy & Hold Return: {metrics['buy_hold_return']:.2%}")
print(f"Annualized Strategy 1 Return: {metrics['annualized_strategy_return']:.2%}")
print(f"Annualized Buy & Hold Return: {metrics['annualized_buy_hold_return']:.2%}")
print(f"Annualized Volatility: {metrics['annualized_volatility']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")   

# Mean-Reversion Strategy Performance Summary

## Cumulative Performance
- **Strategy Return**: **-19.56%**
- **Buy & Hold Return**: **+98.52%**

The mean-reversion strategy lost value over the full period, while simply holding the S&P 500 index would have nearly doubled the investment.

---

## Annualized Metrics
- **Annualized Strategy Return**: **+95.73%**
- **Annualized Buy & Hold Return**: **+114.75%**
- **Annualized Volatility (Strategy)**: **20.21%**
- **Sharpe Ratio (Strategy)**: **4.74**

Despite the negative total return, the annualized return appears misleadingly high due to high early gains.  
The Sharpe ratio is also distorted - likely inflated by volatility clustering or early gains, not by sustained outperformance.

---

## Risk Metric
- **Max Drawdown (Strategy)**: **-45.78%**

This means the strategy lost nearly half of its value from peak to trough. A very steep and risky profile.

---

## Interpretation and Insights
- The strategy started well but failed to recover from large losses, especially during strong directional market trends, which tend to hurt mean-reverting strategies.
- The mean-reversion assumption may not hold consistently in the S&P 500, particularly during trending markets.
- The high Sharpe ratio here is not reliable without further validation - it is likely the result of early variance and not robust performance.
